In [1]:
import os
from os.path import join
from scipy.io import loadmat
from glob import glob

def load_snkth(root: str = "data/SNKTH"):
    """
    Load SNKTH dataset: 6-class event dataset with .mat files per sample.
    Assumes each class is a subfolder in `root`, with .mat files inside.

    Returns:
        events_list: list of dicts with keys ['x', 'y', 't', 'p']
        labels: list of integer class labels
    """
    class_names = sorted(os.listdir(root))  # ['class1', 'class2', ...]
    label_map = {cls_name: i for i, cls_name in enumerate(class_names)}

    events_list = []
    labels = []

    for cls in class_names:
        folder = join(root, cls)
        mat_files = glob(join(folder, "*.mat"))
        for path in mat_files:
            mat = loadmat(path)

            # Expecting keys like mat['x'], mat['y'], mat['t'], mat['p']
            # Adjust these keys if your .mat structure is different
            try:
                x = mat["x"].squeeze()
                y = mat["y"].squeeze()
                t = mat["t"].squeeze()
                p = mat["p"].squeeze()
            except KeyError:
                print(f"[WARN] Skipped {path} (missing keys)")
                continue

            events = {
                "x": x.astype(int),
                "y": y.astype(int),
                "t": t.astype(int),
                "p": p.astype(int)
            }

            events_list.append(events)
            labels.append(label_map[cls])

    print(f"[LOG] Loaded SNKTH: {len(events_list)} samples across {len(class_names)} classes")
    return events_list, labels


In [2]:
def ev_loader_snkth(root="data/SNKTH", dataset="full"):
    events, labels = load_snkth(root)
    print("LOG: Loaded SNKTH dataset")
    return events, labels


In [3]:
events, l = ev_loader_snkth()

[LOG] Loaded SNKTH: 9472 samples across 12 classes
LOG: Loaded SNKTH dataset


In [6]:
# print(events)
print(events[0])
# print(events[0][0])
# print(events[0][0][0])

{'x': array([ 69,  69,  72, ...,  89, 103,  91]), 'y': array([103, 101, 102, ...,  16,  41,  89]), 't': array([ 48424,  48899,  49147, ..., 999461, 999461, 999968]), 'p': array([0, 0, 1, ..., 0, 0, 1])}


In [2]:
import os
import numpy as np
from scipy.io import loadmat

import numpy as np
from scipy.io import loadmat

def read_snkth_mat_int(path):
    mat_data = loadmat(path)

    # Automatically find the relevant data key
    data_key = next((k for k in mat_data.keys() if not k.startswith("__")), None)
    if data_key is None:
        raise KeyError(f"No usable data found in {path}")

    raw = mat_data[data_key]  # e.g., shape (N, 4) or structured array

    # Check if it's packed event data (assumed 4 columns: t, _, ...), or already split
    if raw.shape[1] == 4 and isinstance(raw, np.ndarray):
        # Example: t, _, other1, other2
        packed = raw[:, 1].astype(np.int32)  # assuming 2nd col is "_"

        x = np.bitwise_and(packed, 16383).astype(np.int16)
        y = np.right_shift(np.bitwise_and(packed, 268419072), 14).astype(np.int16)
        p = np.right_shift(np.bitwise_and(packed, 268435456), 28).astype(np.int16)

        dtype = [('t', 'i4'), ('x', 'i2'), ('y', 'i2'), ('p', 'i2')]
        result = np.empty(len(packed), dtype=dtype)
        result['t'] = raw[:, 0].astype(np.int32)
        result['x'] = x
        result['y'] = y
        result['p'] = p
        return result

    elif {'x', 'y', 'p', 't'}.issubset(set(mat_data.keys())):
        # already unpacked fields
        x = mat_data['x'].squeeze().astype(np.int16)
        y = mat_data['y'].squeeze().astype(np.int16)
        p = mat_data['p'].squeeze().astype(np.int16)
        t = mat_data['t'].squeeze().astype(np.int32)

        dtype = [('t', 'i4'), ('x', 'i2'), ('y', 'i2'), ('p', 'i2')]
        result = np.empty(len(t), dtype=dtype)
        result['t'] = t
        result['x'] = x
        result['y'] = y
        result['p'] = p
        return result

    else:
        raise ValueError("Unknown structure of .mat file; expected packed or x/y/p/t fields")


class SNKTH():
    def __init__(self, root, split="train"):
        """
        Initialize SNKTH dataset by collecting all .mat files and their labels.
        Assumes `root/split/` contains subfolders for each class, similar to NCARS.
        """
        self.files = []
        # Replace ["classA", "classB", ...] with actual class folder names in SNKTH dataset:
        class_folders = ["jogging", "running"]  # example class names
        for label, sub in enumerate(class_folders):
            folder = os.path.join(root,  sub)
            for fname in os.listdir(folder):
                if fname.lower().endswith(".mat"):
                    filepath = os.path.join(folder, fname)
                    self.files.append((filepath, label))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        path, label = self.files[idx]
        data = read_snkth_mat_int(path)   # load the .mat file data (events or features)
        return data, label


In [3]:
train_dataset = SNKTH(root="data/SNKTH")
X_train_list = []
y_train_list = []
for i in range(len(train_dataset)):
    features, label = train_dataset[i]
    X_train_list.append(features.flatten())  # flatten in case features are not 1D
    y_train_list.append(label)


In [13]:
e, l = train_dataset[1]
print(e, l)

print(" number of samples = ", len(X_train_list))
print(" number of samples = ", len(X_train_list))

print(len(train_dataset[4]))
print(len(train_dataset[0]))
print(len(train_dataset[0][0]))
print(len(train_dataset[0][0][0]))
print(train_dataset[-1][0][-1])
print(train_dataset[0][0][1])
print(train_dataset[0][0][-1])

[( 45117,  74, 75, 1) ( 45306,  75, 32, 1) ( 45418,  74, 76, 1) ...
 (998506, 157,  9, 1) (999461, 158, 19, 0) (999461, 159, 16, 1)] 0
 number of samples =  2147
 number of samples =  2147
2
2
7970
4
(999952, 24, 9, 0)
(292105, 159, 63, 0)
(999877, 120, 26, 1)
